# Testing lightcurve plots

In [5]:
#!/usr/bin/env python3
# type: ignore
import os
import re
import sys
import time
from io import StringIO

import pandas as pd
import requests

BASEURL = "https://fallingstar-data.com/forcedphot"
# BASEURL = "http://127.0.0.1:8000"

if os.environ.get("ATLASFORCED_SECRET_KEY"):
    token = os.environ.get("ATLASFORCED_SECRET_KEY")
    print("Using stored token")
else:
    data = {"username": "JinZheng", "password": "Swagrider2015"}

    resp = requests.post(url=f"{BASEURL}/api-token-auth/", data=data)

    if resp.status_code == 200:
        token = resp.json()["token"]
        print(f"Your token is {token}")
        print("Store this by running/adding to your .zshrc file:")
        print(f'export ATLASFORCED_SECRET_KEY="{token}"')
    else:
        print(f"ERROR {resp.status_code}")
        print(resp.text)
        sys.exit()


headers = {"Authorization": f"Token {token}", "Accept": "application/json"}

task_url = None
while not task_url:
    with requests.Session() as s:
        # alternative to token auth
        # s.auth = ('USERNAME', 'PASSWORD')
        resp = s.post(f"{BASEURL}/queue/", headers=headers, data={"ra": 314, "dec": 14, "mjd_min": 60581.44075})

        if resp.status_code == 201:  # successfully queued
            task_url = resp.json()["url"]
            print(f"The task URL is {task_url}")
        elif resp.status_code == 429:  # throttled
            message = resp.json()["detail"]
            print(f"{resp.status_code} {message}")
            t_sec = re.findall(r"available in (\d+) seconds", message)
            t_min = re.findall(r"available in (\d+) minutes", message)
            if t_sec:
                waittime = int(t_sec[0])
            elif t_min:
                waittime = int(t_min[0]) * 60
            else:
                waittime = 10
            print(f"Waiting {waittime} seconds")
            time.sleep(waittime)
        else:
            print(f"ERROR {resp.status_code}")
            print(resp.text)
            sys.exit()


result_url = None
taskstarted_printed = False
while not result_url:
    with requests.Session() as s:
        resp = s.get(task_url, headers=headers)

        if resp.status_code == 200:  # HTTP OK
            if resp.json()["finishtimestamp"]:
                result_url = resp.json()["result_url"]
                print(f"Task is complete with results available at {result_url}")
            elif resp.json()["starttimestamp"]:
                if not taskstarted_printed:
                    print(f"Task is running (started at {resp.json()['starttimestamp']})")
                    taskstarted_printed = True
                time.sleep(2)
            else:
                print(f"Waiting for job to start (queued at {resp.json()['timestamp']})")
                time.sleep(4)
        else:
            print(f"ERROR {resp.status_code}")
            print(resp.text)
            sys.exit()

with requests.Session() as s:
    textdata = s.get(result_url, headers=headers).text

    # if we'll be making a lot of requests, keep the web queue from being
    # cluttered (and reduce server storage usage) by sending a delete operation
    # s.delete(task_url, headers=headers).json()

dfresult = pd.read_csv(StringIO(textdata.replace("###", "")), delim_whitespace=True)
print(dfresult)


Your token is 938191f3499bfce3ac5b3566e298b2c9106d84d3
Store this by running/adding to your .zshrc file:
export ATLASFORCED_SECRET_KEY="938191f3499bfce3ac5b3566e298b2c9106d84d3"
The task URL is https://fallingstar-data.com/forcedphot/queue/1799680/
Task is running (started at 2024-10-28T16:55:33Z)
Task is complete with results available at https://fallingstar-data.com/forcedphot/static/results/job1799680.txt
              MJD       m     dm  uJy  duJy  F  err  chi/N     RA   Dec  \
0    60582.313161 -20.646  0.589  -20    12  c    0   1.08  314.0  14.0   
1    60582.316344 -22.868  3.667   -3    10  c    0   1.19  314.0  14.0   
2    60582.321868 -22.196  2.548   -5    12  c    0   0.91  314.0  14.0   
3    60582.331818  22.720  3.286    3    10  c    0   2.10  314.0  14.0   
4    60582.335527  20.487  0.547   23    13  o    0   1.32  314.0  14.0   
..            ...     ...    ...  ...   ... ..  ...    ...    ...   ...   
145  60608.778900 -21.294  1.125  -11    12  c    0   1.17  314

C:\Users\jhzhe\AppData\Local\Temp\ipykernel_23300\4075599804.py:95: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  dfresult = pd.read_csv(StringIO(textdata.replace("###", "")), delim_whitespace=True)
